<a href="https://colab.research.google.com/github/Thanawat2002/colab/blob/main/Deploy_pima_on_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import loadtxt
from keras.models import load_model
import pickle
#load model
pickled_model = pickle.load(open('/content/pima_model.pkl', 'rb'))
#pickled_model.predict(X_test)
#summaize model
pickled_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
import io
import pandas as pd
from google.colab import files

uploaded = files.upload()
udata = pd.read_csv(io.BytesIO(uploaded['pima.csv']))

Saving pima.csv to pima.csv


In [3]:
from sklearn.preprocessing import StandardScaler
X = udata.iloc[:, 0:8]
sc = StandardScaler()
sc.fit(X)

StandardScaler()

In [4]:
pip install flask-ngrok

In [5]:
from google.colab import drive
drive.mount('/content/drive') #ระวังเครื่องหมาย ‘ กับ ' ต่างกันนะคะ

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/MyDrive/Colab Notebooks/web_pima

/content/drive/MyDrive/Colab Notebooks/web_pima


In [7]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np

In [8]:
!pip install pyngrok

In [9]:
!ngrok authtoken 2U3gy5LVLYWVaLhAGL1ObRjjjm9_4Cu5Pbt1mxn7Q2nK95xMW

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:
from keras.models import load_model

app = Flask(__name__)

run_with_ngrok(app)

model = pickle.load(open('/content/pima_model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=["POST"])

def predict():
    if request.method == 'POST':

        nt = request.form['nt']
        plasma_glucose = request.form['plasma_glucose']
        db_pressure = request.form['db_pressure']
        ts_thickness = request.form['ts_thickness']
        serum = request.form['serum']
        boday_mass_index = request.form['boday_mass_index']
        dp_function = request.form['dp_function']
        age = request.form['age']

        sample_data = [nt, plasma_glucose, db_pressure, ts_thickness, serum, boday_mass_index, dp_function, age]
        # sample_data = [10, 139, 80, 0, 0, 27.1, 1.441, 57] #class 0 ไม่เป็น
        # sample_data = [1, 89, 66, 23, 94, 28.1, 0.167, 21] #class 0 ไม่เป็น
        # sample_data = [6, 148, 72, 35, 0, 33.6, 0.627, 50] #class 1 เป็น
        # sample_data = [5, 166, 72, 19, 175, 25.8, 0.587, 51] #class 1 เป็น
        # print(sample_data)
        clean_data = [float(i) for i in sample_data] # แปลง str to float
        final_input = np.array(clean_data).reshape(1,-1)
        # print(final_input)
        input_final = sc.transform(final_input)

        prediction = model.predict(input_final)
        result = (prediction > 0.5).astype("int32")
        print("predictedclass ",result)

    return render_template('index.html',result_prediction=result)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://10ea-34-73-165-23.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Oct/2023 16:17:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Oct/2023 16:17:24] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 70ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [12/Oct/2023 16:18:10] "POST /predict HTTP/1.1" 200 -


predictedclass  [[1]]
1/1 [==============================] - 0s 23ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [12/Oct/2023 16:19:10] "POST /predict HTTP/1.1" 200 -


predictedclass  [[0]]
